In [ ]:
import numpy as np
import cv2
from keras.preprocessing import image
from keras.models import model_from_json


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

#face expression recognizer initialization
model = model_from_json(open("facial_expression_model_structure.json", "r").read())
model.load_weights('facial_expression_model_weights.h5') #load weights

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

while(True):
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        #draw rectangle to main image
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
        #crop detected face
        detected_face = img[int(y):int(y+h), int(x):int(x+w)] 
        #transform to gray scale
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)
        #resize to 48x48
        detected_face = cv2.resize(detected_face, (48, 48)) 
        
        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        # normalize all pixels in scale of [0, 1]
        img_pixels /= 255 
        #store probabilities of 7 expressions
        predictions = model.predict(img_pixels) 
        
        #find max indexed array 0: angry, 1:disgust, 2:fear, 3:happy, 4:sad, 5:surprise, 6:neutral
        max_index = np.argmax(predictions[0])
        
        emotion = emotions[max_index]
        
        #write emotion text above rectangle
        cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
    cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break

cap.release()
cv2.destroyAllWindows()